<a href="https://colab.research.google.com/github/narfdf999/Arquitectura/blob/master/PMV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Primera aproximación al problema de generar edificio a lo largo de un camino dibujado por el arquitecto

Importar librerías

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.layers import Dense, Dropout, Input
from keras.models import Model,Sequential
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import adam

Preparación y carga de datos

Optimizador ADAM


In [0]:
def adam_optimizer():
    return Adam(lr=0.0002, beta_1=0.5)

Generador

Discriminador

GAN

Postprocesado de datos de salida

Entrenamiento